In [ ]:
import numpy as np
import csv
import torch
import pandas as pd
import torch.nn as nn
import os
import heapq
from tqdm import tqdm
import torch.optim as optim
import random
import torch.nn.functional as F
import warnings
import wandb
import math
from torch.nn.utils import clip_grad_norm_
warnings.filterwarnings("ignore")

In [ ]:
val=torch.cuda.is_available()
if val == 1:
    device= torch.device('cuda')
else:
    device = torch.device('gpu')
    
print(device)

In [ ]:
# !wandb login apni_api

In [ ]:
def loadData(params):
    dataset_path = params['dataset_path']
    train_data = csv.reader(open(dataset_path + '/hin/hin_train.csv',encoding='utf8'))
    val_data = csv.reader(open(dataset_path + '/hin/hin_valid.csv',encoding='utf8'))
    test_data = csv.reader(open(dataset_path + '/hin/hin_test.csv',encoding='utf8'))
    train_translations = []
    test_words=[]
    val_translations = []
    val_words=[]
    train_words =[]
    test_translations = []
    pad=''
    start='$'
    end ='&' 
    train_data_list = list(train_data)
    train_len = len(train_data_list)
    i = 0
    while i < train_len:
        pair = train_data_list[i]
        train_words.append(pair[0] + end)
        train_translations.append(start + pair[1] + end)
        i += 1  
    
    i=0
    val_data_list = list(val_data)
    val_len = len(val_data_list)
    while i < val_len :
        pair=val_data_list[i]
        val_words.append(pair[0]+end)
        val_translations.append(start+pair[1]+end)
        i+=1
        
    i=0
    test_data_list = list(test_data)
    test_len = len(test_data_list)
    while i < test_len :
        pair=test_data_list[i]
        test_words.append(pair[0]+end)
        test_translations.append(start+pair[1]+end)
        i+=1   
        
 
    
    test_words =np.array(test_words)
    train_translations = np.array(train_translations)
    val_translations =np.array(val_translations)
    train_words = np.array(train_words)
    test_translations = np.array(test_translations)
    val_words =np.array(val_words)
    
    
    
    output_vocab,input_vocab = set() , set()
    i = 0
    word_len=len(train_words)
    while i < word_len :
        word = train_words[i]
        character_index = 0
        while character_index < len(word):
            character = word[character_index]
            input_vocab.add(character)
            character_index += 1
        i += 1         
    
    
    i = 0
    word_len=len(val_words)
    while i < word_len :
        word = val_words[i]
        character_index = 0
        while character_index < len(word):
            character = word[character_index]
            input_vocab.add(character)
            character_index += 1
        i += 1
    
    i = 0
    word_len=len(test_words)
    while i < word_len :
        word = test_words[i]
        character_index = 0
        while character_index < len(word):
            character = word[character_index]
            input_vocab.add(character)
            character_index += 1
        i += 1

    i = 0
    word_len=len(train_translations)
    while i < word_len :
        word = train_translations[i]
        character_index = 0
        while character_index < len(word):
            character = word[character_index]
            output_vocab.add(character)
            character_index += 1
        i += 1
        
    i = 0
    word_len=len(val_translations)
    while i < word_len :
        word = val_translations[i]
        character_index = 0
        while character_index < len(word):
            character = word[character_index]
            output_vocab.add(character)
            character_index += 1
        i += 1
        
    i = 0
    word_len=len(test_translations)
    while i < word_len :
        word = test_translations[i]
        character_index = 0
        while character_index < len(word):
            character = word[character_index]
            output_vocab.add(character)
            character_index += 1
        i += 1
    
    output_vocab.remove(start)
    input_vocab.remove(end)
    output_vocab.remove(end)
    
    output_vocab= [pad, start, end] + list(sorted(output_vocab))
    input_vocab = [pad, start, end] + list(sorted(input_vocab))
            
 
    output_index,input_index = {char: idx for idx, char in enumerate(output_vocab)},{char: idx for idx, char in enumerate(input_vocab)}
    output_index_rev,input_index_rev = {idx: char for char, idx in output_index.items()},{idx: char for char, idx in input_index.items()}
    

    max_len = max(max([len(word) for word in np.hstack((train_words, test_words, val_words))]), max([len(word) for word in np.hstack((train_translations, val_translations, test_translations))]))
        
    preprocessed_data = {
        'SOS' : start,
        'EOS' : end,
        'PAD' : pad,
        'train_words' : train_words,
        'train_translations' : train_translations,
        'val_words' : val_words,
        'val_translations' : val_translations,
        'test_words' : test_words,
        'test_translations' : test_translations,
        'max_enc_len' : max([len(word) for word in np.hstack((train_words, test_words, val_words))]),
        'max_dec_len' : max([len(word) for word in np.hstack((train_translations, val_translations, test_translations))]),
        'max_len' : max_len,
        'input_index' : input_index,
        'output_index' : output_index,
        'input_index_rev' : input_index_rev,
        'output_index_rev' : output_index_rev
    }
    return preprocessed_data

In [ ]:
def create_tensor(preprocessed_data):
    prop_data=preprocessed_data['max_len']
    leng=len(preprocessed_data['train_words'])
    d_type='int64'
    input_data = np.zeros((prop_data,leng), dtype = d_type)
    output_data = np.zeros((prop_data,leng), dtype = d_type)
    leng=len(preprocessed_data['val_words'])
    val_input_data = np.zeros((prop_data,leng), dtype = d_type)
    val_output_data = np.zeros((prop_data,leng), dtype = d_type)
    leng=len(preprocessed_data['test_words'])
    test_input_data = np.zeros((prop_data,leng), dtype = d_type)
    test_output_data = np.zeros((prop_data,leng), dtype = d_type)
    

    idx = 0
    while idx < len(preprocessed_data['train_words']):
        w = preprocessed_data['train_words'][idx]
        t = preprocessed_data['train_translations'][idx]

        i = 0
        while i < len(w):
            char = w[i]
            input_data[i, idx] = preprocessed_data['input_index'][char]
            i += 1

        i = 0
        while i < len(t):
            char = t[i]
            output_data[i, idx] = preprocessed_data['output_index'][char]
            i += 1
        idx += 1            
        

            
    idx = 0
    while idx < len(preprocessed_data['val_words']):
        w = preprocessed_data['val_words'][idx]
        t = preprocessed_data['val_translations'][idx]

        i = 0
        while i < len(w):
            char = w[i]
            val_input_data[i, idx] = preprocessed_data['input_index'][char]
            i += 1

        i = 0
        while i < len(t):
            char = t[i]
            val_output_data[i, idx] = preprocessed_data['output_index'][char]
            i += 1
        idx += 1            
        
            
    idx = 0
    while idx < len(preprocessed_data['test_words']):
        w = preprocessed_data['test_words'][idx]
        t = preprocessed_data['test_translations'][idx]

        i = 0
        while i < len(w):
            char = w[i]
            test_input_data[i, idx] = preprocessed_data['input_index'][char]
            i += 1

        i = 0
        while i < len(t):
            char = t[i]
            test_output_data[i, idx] = preprocessed_data['output_index'][char]
            i += 1
        idx += 1            
        
            
            
    output_data=torch.tensor(output_data, dtype = torch.int64)
    input_data = torch.tensor(input_data,dtype = torch.int64)
    val_output_data=torch.tensor(val_output_data, dtype = torch.int64)
    val_input_data = torch.tensor(val_input_data,dtype = torch.int64)
    test_output_data=torch.tensor(test_output_data, dtype = torch.int64)
    test_input_data= torch.tensor(test_input_data,dtype = torch.int64)
    
    tensors = {
        'input_data' : input_data,
        'output_data' : output_data,
        'val_input_data' : val_input_data,
        'val_output_data' : val_output_data, 
        'test_input_data' : test_input_data,
        'test_output_data' : test_output_data
    }
    return tensors

In [ ]:
class Encoder(nn.Module): ####################################
    def __init__(self, params, preprocessed_data):
        super(Encoder, self).__init__()
        self.cell_type = params['cell_type']
        self.dropout = nn.Dropout(params['dropout'])
        self.embedding = nn.Embedding(len(preprocessed_data['input_index']), params['embedding_size'])
        if self.cell_type == 'RNN':
            self.cell = nn.RNN(params['embedding_size'], params['hidden_size'], params['num_layers_enc'], dropout = params['dropout'], bidirectional = params['bi_dir'])
        elif self.cell_type == 'GRU':
            self.cell = nn.GRU(params['embedding_size'], params['hidden_size'], params['num_layers_enc'], dropout = params['dropout'], bidirectional = params['bi_dir'])
        
    def forward(self, x):
        drop_par = self.embedding(x)
        _ , hidden = self.cell(self.dropout(drop_par))
        return hidden

In [ ]:
class Decoder(nn.Module):####################################
    def __init__(self, params, preprocessed_data):
        super(Decoder, self).__init__()
        self.cell_type = params['cell_type']
        self.dropout = nn.Dropout(params['dropout'])
        self.embedding = nn.Embedding(len(preprocessed_data['output_index']), params['embedding_size'])
        if self.cell_type == 'RNN':
            self.cell = nn.RNN(params['embedding_size'], params['hidden_size'], params['num_layers_dec'], dropout = params['dropout'], bidirectional = params['bi_dir'])
        elif self.cell_type == 'GRU':
            self.cell = nn.GRU(params['embedding_size'], params['hidden_size'], params['num_layers_dec'], dropout = params['dropout'], bidirectional = params['bi_dir'])
        self.fc = nn.Linear(params['hidden_size'] * 2 if params['bi_dir'] == True else params['hidden_size'], len(preprocessed_data['output_index']))

    def forward(self, x, hidden, cell):
        embedding = self.embedding(x.unsqueeze(0))
        outputs, hidden = self.cell(self.dropout(embedding), hidden)
        predictions = self.fc(outputs).squeeze(0)
        return predictions, hidden

In [ ]:
class Seq2Seq(nn.Module):####################################
    def __init__(self, encoder, decoder, params,  preprocessed_data):
        super(Seq2Seq, self).__init__()
        self.cell_type = params['cell_type']
        self.decoder, self.encoder  = decoder, encoder
        self.output_index_len = len(preprocessed_data['output_index'])
        self.tfr = params['teacher_fr']

    def forward(self, source, target):
        batch_size, target_len = source.shape[1], target.shape[0]
        x = target[0]
        outputs = torch.zeros(target_len, batch_size, self.output_index_len).to(device)
        hidden = self.encoder(source)
        for t in range(1, target_len):
            output, hidden = self.decoder(x, hidden, None)
            outputs[t], best_guess = output, output.argmax(1)
            x = best_guess if random.random() >= self.tfr else target[t]
        return outputs